---
title: "Create training dataset from online feature store enabled feature groups"
date: 2021-04-25
type: technical_note
draft: false
---

### Establish a connection with your Hopsworks feature store.

In [1]:
import hsfs
connection = hsfs.connection()
# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
30,application_1624276539905_0010,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

## Get feature groups

In [2]:
card_transactions_10m_agg = fs.get_feature_group("card_transactions_10m_agg", version = 1)
card_transactions_1h_agg = fs.get_feature_group("card_transactions_1h_agg", version = 1)
card_transactions_12h_agg = fs.get_feature_group("card_transactions_12h_agg", version = 1)

## Create training dataset

In [5]:
query = card_transactions_10m_agg.select(["stdev_amt_per_10m", "avg_amt_per_10m", "num_trans_per_10m"])\
                                 .join(card_transactions_1h_agg.select(["stdev_amt_per_1h", "avg_amt_per_1h", "num_trans_per_1h"]))\
                                 .join(card_transactions_12h_agg.select(["stdev_amt_per_12h", "avg_amt_per_12h", "num_trans_per_12h"]))


In [11]:
dir(query)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_feature_store_id', '_feature_store_name', '_filter', '_hive_engine', '_hopsworks_json', '_joins', '_left_feature_group', '_left_feature_group_end_time', '_left_feature_group_start_time', '_left_features', '_query_constructor_api', '_register_hudi_tables', '_register_on_demand', '_storage_connector_api', 'as_of', 'filter', 'join', 'json', 'left_feature_group_end_time', 'left_feature_group_start_time', 'pull_changes', 'read', 'show', 'to_dict', 'to_string']

In [15]:
query.read().count()

200

In [17]:
query.show(20)

+------------------+---------------+-----------------+------------------+------------------+----------------+------------------+------------------+-----------------+
| stdev_amt_per_10m|avg_amt_per_10m|num_trans_per_10m|  stdev_amt_per_1h|    avg_amt_per_1h|num_trans_per_1h| stdev_amt_per_12h|   avg_amt_per_12h|num_trans_per_12h|
+------------------+---------------+-----------------+------------------+------------------+----------------+------------------+------------------+-----------------+
|               NaN|          577.2|                1|261.07835164946175|            196.75|               3|1709.5281368238423| 663.3857894736843|               19|
|               NaN|          35.63|                1|3235.3105087765534|2817.2200000000003|               2| 349.1966821397238|224.18000000000004|               24|
|               NaN|         979.52|                1|               NaN|           5667.57|               1|1442.8493355008477| 554.5813333333333|               15|
|   

In [18]:
td_meta = fs.create_training_dataset(name="card_fraud_model_holuser20_csv",
                               description="Training dataset to train card fraud model",
                               #data_format="tfrecord",
                               data_format="csv", 
                               version=1)

td_meta.save(query)

An error was encountered:
Metadata operation error: (url: https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/1145/featurestores/1093/trainingdatasets/1039/statistics). Server response: 
HTTP code: 400, HTTP reason: Bad Request, error code: 270137, error msg: Error saving statistics, user msg: Not a valid JSON
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/training_dataset.py", line 156, in save
    self._statistics_engine.compute_statistics(self, self.read())
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/core/statistics_engine.py", line 69, in compute_statistics
    self._statistics_api.post(metadata_instance, stats)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/core/statistics_api.py", line 46, in post
    "POST", path_params, headers=headers, data=statistics.json()
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hsfs/decorators.py",

In [10]:
card_transactions_10m_agg.select(["stdev_amt_per_10m", "avg_amt_per_10m", "num_trans_per_10m"]).show(10)

+-----------------+---------------+-----------------+
|stdev_amt_per_10m|avg_amt_per_10m|num_trans_per_10m|
+-----------------+---------------+-----------------+
|              NaN|          82.28|                1|
|              NaN|          12.77|                1|
|              NaN|          88.43|                1|
|              NaN|          32.22|                1|
|              NaN|          42.09|                1|
|              NaN|         459.94|                1|
|              NaN|          95.11|                1|
|              NaN|        1308.17|                1|
|              NaN|         776.22|                1|
|              NaN|          92.62|                1|
+-----------------+---------------+-----------------+
only showing top 10 rows